### Machine Learning and Contect Analytics

MSc in Business Analytics

Department of Managment Science & Technology

Athens University of Economics and Business

#### Authors
* Alvanou Marianna, ΑΜ: p2822201
* Vlachakis Sotirios, ΑΜ: p2822228

# Model 2 ~ Greek Bert

In [1]:
# import tensorflow as tf

# # Set GPU memory growth to limit memory usage
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00


In [3]:
!pip install nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [5]:
%matplotlib inline

import tensorflow as tf
import nlp
import random

import pandas as pd
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.core.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
# df = pd.read_excel("C:\\Users\\mario\\Downloads\\data_cleaning.xlsx", usecols = ['Brand', 'Comments', 'NPS_Label'])

# df.head(5)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
file_path = "/content/drive/MyDrive/Colab Notebooks/data_cleaning.xlsx"
df = pd.read_excel(file_path)

In [11]:
df = df[df['NPS_Label'] != 'Neutral']

In [12]:
def nps_new(nps):
        if nps == 'Positive':
            return 0
        else:
            return 1

In [13]:
# Net Promoter Score

df['NPS_encoded'] = df['NPS_Label'].apply(nps_new)

<ipython-input-13-ee3b716c64ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NPS_encoded'] = df['NPS_Label'].apply(nps_new)


In [14]:
df.sample(5)

,Unnamed: 0,Brand,Comments,NPS_Label,NPS_encoded
10153,17835,10,αμεση εξυπηρετηση!,Positive,0
5539,10014,10,"ευγένεια, άμεσες και έγκυρες συμβουλές.",Positive,0
11206,19529,9,το γεγονοσ οτι λυθηκε το προβλημα μου .,Positive,0
6330,11357,10,αλλ is perfect!!! τhank you very much for aλλ!!!,Positive,0
1789,3589,10,πολυ ευγενικη και εξυπηρετικη η υπαλληλοσ,Positive,0


In [15]:
# Check the class distribution after oversampling
print(df['NPS_Label'].value_counts())

Positive    11453
Negative     1952
Name: NPS_Label, dtype: int64


In [16]:
df.dtypes

Unnamed: 0      int64
Brand           int64
Comments       object
NPS_Label      object
NPS_encoded     int64
dtype: object

In [17]:
df['Comments'] = df['Comments'].astype(str)

<ipython-input-17-de0b05d39e11>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Comments'] = df['Comments'].astype(str)


#### Original Data - without some technic

In [18]:
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [19]:
# Load the BERT tokenizer for Greek
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")

# Load the pre-trained BERT model for sequence classification (adjust num_labels as needed)
model = TFAutoModelForSequenceClassification.from_pretrained("nlpaueb/bert-base-greek-uncased-v1", num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/bert-base-greek-uncased-v1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
#Prepair the data

from sklearn.model_selection import train_test_split

X= list(df['Comments'])
y= np.array(df['NPS_encoded'])

# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.20, random_state=42, stratify=y_train_val)

In [21]:
import numpy as np

# Tokenize the training data
train_encodings = tokenizer(list(X_train), return_tensors='tf', padding=True, truncation=True, max_length=256)
train_inputs = {
    'input_ids': np.array(train_encodings['input_ids']),
    'token_type_ids': np.array(train_encodings['token_type_ids']),
    'attention_mask': np.array(train_encodings['attention_mask'])
}
train_labels = np.array(y_train)

# Tokenize the validation data
val_encodings = tokenizer(list(X_val), return_tensors='tf', padding=True, truncation=True, max_length=256)
val_inputs = {
    'input_ids': np.array(val_encodings['input_ids']),
    'token_type_ids': np.array(val_encodings['token_type_ids']),
    'attention_mask': np.array(val_encodings['attention_mask'])
}
val_labels = np.array(y_val)

# Tokenize the test data
test_encodings = tokenizer(list(X_test), return_tensors='tf', padding=True, truncation=True, max_length=256)
test_inputs = {
    'input_ids': np.array(test_encodings['input_ids']),
    'token_type_ids': np.array(test_encodings['token_type_ids']),
    'attention_mask': np.array(test_encodings['attention_mask'])
}
test_labels = np.array(y_test)

In [22]:
from tensorflow.keras.utils import to_categorical

train_labels_onehot = to_categorical(y_train, num_classes=2)
val_labels_onehot = to_categorical(y_val, num_classes=2)
test_labels_onehot = to_categorical(y_test, num_classes=2)

In [23]:
# Fine-tuning the BERT model on the training data with
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping callback
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]
history = model.fit(train_inputs, train_labels_onehot,
                    validation_data=(val_inputs, val_labels_onehot),
                    epochs=10, callbacks=callbacks)

Epoch 1/10
285/285 [==============================] - 539s 2s/step - loss: 11.5499 - accuracy: 0.8170 - val_loss: 13.7700 - val_accuracy: 0.8543
Epoch 2/10
285/285 [==============================] - 462s 2s/step - loss: 12.4241 - accuracy: 0.8544 - val_loss: 13.7700 - val_accuracy: 0.8543
Epoch 3/10
285/285 [==============================] - 475s 2s/step - loss: 12.4206 - accuracy: 0.8544 - val_loss: 13.7700 - val_accuracy: 0.8543


In [24]:
 # Evaluate the model on the test set
loss, accuracy = model.evaluate(test_inputs, test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

ValueError: ignored

### Plot Loss and Accuracy

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

#### Oversampling Technic

In [ ]:
#Oversampling Technic

import numpy as np
from imblearn.over_sampling import RandomOverSampler

# Example DataFrame
data = pd.DataFrame(df)

# Extract features (text) and labels (sentiment)
x_o = data['Comments']
y_o = data['NPS_Label']

# Create the RandomOverSampler object
oversampler = RandomOverSampler(random_state=42)

# Resample the data
x_over, y_over = oversampler.fit_resample(x_o.values.reshape(-1, 1), y)

# Create a new DataFrame with the oversampled data
df_over = pd.DataFrame({'Comments': x_over.flatten(), 'NPS_Label': y_over})

# Check the class distribution after oversampling
print(df_over['NPS_Label'].value_counts())

In [ ]:
def nps_new(nps):
        if nps == 'Positive':
            return 0
        else:
            return 1

In [ ]:
# Net Promoter Score
df_over = df_over[df_over['NPS_Label'] != 'Neutral']

df_over['NPS_encoded'] = df_over['NPS_Label'].apply(nps_new)

In [ ]:
df_over.sample(5)

In [ ]:
# Load the BERT tokenizer for Greek
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")

# Load the pre-trained BERT model for sequence classification (adjust num_labels as needed)
model = TFAutoModelForSequenceClassification.from_pretrained("nlpaueb/bert-base-greek-uncased-v1", num_labels=3)

In [ ]:
#Prepair the data

from sklearn.model_selection import train_test_split

x_o= list(df_over['Comments'])
y_o= np.array(df_over['NPS_encoded'])

# Split the data into training, validation, and test sets
X_train_val_o, X_test_o, y_train_val_o, y_test_o = train_test_split(x_o, y_o, test_size=0.15, random_state=42, stratify=y_o)
X_train_o, X_val_o, y_train_o, y_val_o = train_test_split(X_train_val_o, y_train_val_o, test_size=0.20, random_state=42, stratify=y_train_val_o)

In [ ]:

# Tokenize the training data
train_encodings_o = tokenizer(list(X_train_o), return_tensors='tf', padding=True, truncation=True, max_length=256)
train_inputs_o = {
    'input_ids': np.array(train_encodings['input_ids']),
    'token_type_ids': np.array(train_encodings['token_type_ids']),
    'attention_mask': np.array(train_encodings['attention_mask'])
}
train_labels_o = np.array(y_train_o)

# Tokenize the validation data
val_encodings_o = tokenizer(list(X_val_o), return_tensors='tf', padding=True, truncation=True, max_length=256)
val_inputs_o = {
    'input_ids': np.array(val_encodings_o['input_ids']),
    'token_type_ids': np.array(val_encodings_o['token_type_ids']),
    'attention_mask': np.array(val_encodings_o['attention_mask'])
}
val_labels_o = np.array(y_val_o)

# Tokenize the test data
test_encodings_o = tokenizer(list(X_test_o), return_tensors='tf', padding=True, truncation=True, max_length=256)
test_inputs_o = {
    'input_ids': np.array(test_encodings_o['input_ids']),
    'token_type_ids': np.array(test_encodings_o['token_type_ids']),
    'attention_mask': np.array(test_encodings_o['attention_mask'])
}
test_labels_o = np.array(y_test_o)

In [ ]:
from tensorflow.keras.utils import to_categorical

train_labels_onehot = to_categorical(y_train, num_classes=2)
val_labels_onehot = to_categorical(y_val, num_classes=2)
test_labels_onehot = to_categorical(y_test, num_classes=2)

In [ ]:
# Fine-tuning the BERT model on the training data with
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping callback
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]
history_o = model_o.fit(train_inputs_o, train_labels_o, validation_data=(val_inputs_o, val_labels_o), epochs=10, callbacks=callbacks)

In [ ]:
 # Evaluate the model on the test set
loss, accuracy = model.evaluate(test_inputs_o, test_labels_o)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

### Plot Loss and Accuracy

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history_o.history['accuracy'], label='Training Accuracy')
plt.plot(history_o.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history_o.history['loss'], label='Training Loss')
plt.plot(history_o.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


### Undersampling Technic

In [ ]:
#Undersampling Technic

from imblearn.under_sampling import RandomUnderSampler

# Example DataFrame
data = pd.DataFrame(df)

# Extract features (text) and labels (sentiment)
x_u = data['Comments']
y_u = data['NPS_Label']

# Create the RandomOverSampler object
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# Resample the data
x_under, y_under = undersampler.fit_resample(x_u.values.reshape(-1, 1), y)

# Create a new DataFrame with the oversampled data
df_under = pd.DataFrame({'Comments': x_under.flatten(), 'NPS_Label': y_under})

# Check the class distribution after oversampling
print(df_under['NPS_Label'].value_counts())

In [ ]:
def nps_new(nps):
        if nps == 'Positive':
            return 0
        else:
            return 1

In [ ]:
# Net Promoter Score
df_under = df_under[df_under['NPS_Label'] != 'Neutral']

df_under['NPS_encoded'] = df_under['NPS_Label'].apply(nps_new)

In [ ]:
df_under.sample(5)

In [ ]:
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [ ]:
# Load the BERT tokenizer for Greek
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")

# Load the pre-trained BERT model for sequence classification (adjust num_labels as needed)
model = TFAutoModelForSequenceClassification.from_pretrained("nlpaueb/bert-base-greek-uncased-v1", num_labels=2)

In [ ]:
#Prepair the data

from sklearn.model_selection import train_test_split

x_u= list(df_under['Comments'])
y= np.array(df_under['NPS_encoded'])

# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.20, random_state=42, stratify=y_train_val)

In [ ]:
from tensorflow.keras.utils import to_categorical

train_labels_onehot = to_categorical(y_train, num_classes=2)
val_labels_onehot = to_categorical(y_val, num_classes=2)
test_labels_onehot = to_categorical(y_test, num_classes=2)

In [ ]:
import numpy as np

# Tokenize the training data
train_encodings = tokenizer(list(X_train), return_tensors='tf', padding=True, truncation=True, max_length=256)
train_inputs = {
    'input_ids': np.array(train_encodings['input_ids']),
    'token_type_ids': np.array(train_encodings['token_type_ids']),
    'attention_mask': np.array(train_encodings['attention_mask'])
}
train_labels = np.array(y_train)

# Tokenize the validation data
val_encodings = tokenizer(list(X_val), return_tensors='tf', padding=True, truncation=True, max_length=256)
val_inputs = {
    'input_ids': np.array(val_encodings['input_ids']),
    'token_type_ids': np.array(val_encodings['token_type_ids']),
    'attention_mask': np.array(val_encodings['attention_mask'])
}
val_labels = np.array(y_val)

# Tokenize the test data
test_encodings = tokenizer(list(X_test), return_tensors='tf', padding=True, truncation=True, max_length=256)
test_inputs = {
    'input_ids': np.array(test_encodings['input_ids']),
    'token_type_ids': np.array(test_encodings['token_type_ids']),
    'attention_mask': np.array(test_encodings['attention_mask'])
}
test_labels = np.array(y_test)